## Instrucciones para esta prueba
- Renombra esta hoja sustituyendo nombre y apellidos con tus datos.
- Explicar en cada paso, con cuadros de texto, la estrategia seguida para resolverlo.
- Subir a la tarea de moodle el resultado final.

Se ha tomado un **texto** en castellano, se han eliminado las tildes, y han quedado solo caracteres en el siguiente alfabeto

In [182]:
#Descomentar y ejecutar
codigos=[32, 33, 40, 41, 42, 44, 45, 46, 58, 59, 63]+[65..78]+[209]
codigos+=[79..90]+[97..110]+[241] +[111..122]+[161, 171, 187, 191, 10]
alfabeto=unicode(''.join([chr(j) for j in codigos]),"latin1")
print alfabeto
len(alfabeto)

 !()*,-.:;?ABCDEFGHIJKLMNÑOPQRSTUVWXYZabcdefghijklmnñopqrstuvwxyz¡«»¿



70

Se ha cifrado el **texto** con el método de Vigenère encriptando tan solo las letras del juego de caracteres

In [183]:
##Descomentar y ejecutar
A,z=alfabeto.index('A'),alfabeto.index('z')
abc=alfabeto[A:z+1]
abc

u'ABCDEFGHIJKLMN\xd1OPQRSTUVWXYZabcdefghijklmn\xf1opqrstuvwxyz'

Con el texto **encriptado**, y considerando el **alfabeto** completo, se han tomado bloques, de la mayor longitud ${\tt b}$ posible para aplicar RSA con la clave ${\tt (mA,eA,dA)}$ 

In [184]:
##Descomentar y ejecutar
load('claveRSA.py')
mA,eA,dA=claveRSA
claveRSA

(17933812577, 691, 2802924859)

El resultado es la lista ${\tt LbloquesRSA}$ que se carga al ejecutar la siguiente celda

In [185]:
##Descomentar y ejecutar
load('bloquesRSA.py')
len(LbloquesRSA)

158716

Se pide revertir el procedimiento para recuperar el **texto** original.

In [186]:
#utilizamos la función de la semana 12, con la que se desencriptaban los bloques
def DesBloques(lista, b, alfabeto):
    L=len(alfabeto)
    bloques=[]
    for num in lista:
        digitos=Integer(num).digits(L, padto=b)##para que de b dígitos
        digitos.reverse()
        bloques.append(digitos)
    texto=''
    for bloque in bloques:
        texto+=''.join(alfabeto[j] for j in bloque)
    return texto

In [187]:
#además, necesitamos la función que desencripta los bloques RSA usando la clave privada, usando la mayor longitud b: floor(log(nA)/log(L))
def potenciaRSA(x, mA, eA):
    return power_mod(x, eA, mA)

def desencriptaRSA(lista,nA,dA, alfabeto):
    L=len(alfabeto)
    b=floor(log(nA)/log(L))
    desencriptado=[potenciaRSA(x,nA,dA) for x in lista]
    texto=DesBloques(desencriptado,b, alfabeto)
    return texto

In [188]:
#desencriptamos el encriptado RSA, usando el alfabeto completo
encrVigenere=desencriptaRSA(LbloquesRSA, mA, dA, alfabeto)
len(encrVigenere)

793580

In [189]:
#ahora que tenemos el encriptado vigenere calculemos la longitud de la clave (Método de Kasiski)

def McdNgramas(texto,k,m=10):
    u'''
    esta función encuentra las m cadenas de longitud k más 
    repetidas y devuelve el mcd de los saltos entre cada cadena 
    (para hacer más sencillo el return, se devuelve en forma de 
    conjunto, para evitar muchas repeticiones)
    '''
    ltexto=len(texto)
    kgramas=[texto[j:j+k] for j in [0..(ltexto-k)]]
    kgramas_=list(set(kgramas))
    frec_dic=dict(zip(kgramas_,[0]*len(kgramas_)))
    for palabra in kgramas:
        frec_dic[palabra]+=1
    frec_lis=zip(frec_dic.values(),frec_dic.keys())
    frec_lis.sort(reverse=True)
    
    if m<len(frec_lis):
        minimo=m
    else:
        minimo=len(frec_lis)
    
    kgramas=[frec_lis[j][-1] for j in range(minimo)]
    distancias=[]
    s=0#índice en la lista distancias (que contiene las listas de las distancias de cada kgrama)
    for pal in kgramas:
        distancias+=[[]]
        j=texto.find(pal)#encontramos primera ocurrencia
        l=j
        while l!=-1 and j+k<ltexto:
            l=texto[j+k:].find(pal)#buscamos desde la última aparición, el índice que nos de, será la distancia (falta añadir k elementos desde el inicio de la cadena anterior)
            if l!=-1:
                distancias[s].append(l+k)#incluímos en la s-ésima lista de distancias l+k
                j+=k+l#actualizamos j para que contenga el índice de la última aparición encontrada
        s+=1
    mcds=[gcd(lista) for lista in distancias]
    return set(mcds)


In [190]:
McdNgramas(encrVigenere, 3, 10), McdNgramas(encrVigenere, 4, 10), McdNgramas(encrVigenere, 5, 10)

({1, 5}, {1, 5}, {1, 5})

In [191]:
#parece que la clave es de longitud 5, ahora usamos unas funciones para hacer un estudio de las frecuencias en las posiciones 0,1,2,3,4 (mod 5)

def MasFrecuente(cadena, k):
    u'''
    esta función nos dice cuál es la k-ésima 
    letra más frecuente en la cadena: cadena
    '''
    char=list(set(cadena))
    if k>len(char):
        print u'error'
        return None
    frecs=dict(zip(char, [0]*len(char)))
    for letra in cadena:
        frecs[letra]+=1
    FrecCar=zip(frecs.values(),frecs.keys())
    FrecCar.sort(reverse=True)
    return FrecCar[k-1][-1]

In [192]:
#veámos qué resultado nos da esta función una vez ejecutada en nuestro encriptado (con el abecedario solo de caracteres)
pista1=''.join([MasFrecuente(encrVigenere[j::5], 1) for j in range(5)])
pista1 #nos da espacios, por lo que miramos en el segundo más frecuente

u'     '

In [193]:
pista=''.join([MasFrecuente(encrVigenere[j::5], 2) for j in range(5)])
pista

u'KVMJS'

In [194]:
#ahora metemos una función que ENCRIPTA un texto con una clave (Vigenere), y además devuelve la contraclave
def cifradoVigenere(texto, clave, alfabeto):
    u'''
    Esta función espera un texto, una palabra clave y un alfabeto
    y devuelve el texto cifrado (Vigenere) con esta clave y la
    contraclave para deshacer el cifrado
    '''
    if any([(letra not in alfabeto) for letra in clave]):
        print (u'Alguna letra en la clave no es válida')
        return None
    
    len_cla,len_alf = len(clave),len(alfabeto)
    clavenum=[alfabeto.index(letra) for letra in clave]
    contraclave=''.join([alfabeto[-j] for j in clavenum])
    
    encriptado=u''
    for k in [0..(len(texto)-1)]:
        letra=texto[k]
        if letra in alfabeto:
            letra=alfabeto[(alfabeto.index(letra)+clavenum[k%len_cla])%len_alf]
        encriptado+=letra
        
    return encriptado, contraclave

In [195]:
# Ahora sabemos que 'eeeee'(+Vigenere)clave=pista, por lo que clave = pista(+Vigenere)(-'eeeee')
prueba,contraclave=cifradoVigenere('holaaa', 'eeeee', abc)
contraclave #esto es lo opuesto a 'eeeee'

u'WWWWW'

In [196]:
# calculamos lo que hemos dicho antes: pista(+Vigenere)(-'eeeee')
posible_clave,c=cifradoVigenere(pista, contraclave, abc)
posible_clave#tiene sentido que sea grifo

u'grifo'

In [197]:
# ya solo queda usar la clave hallada para obtener la contraclave que nos de el texto final
p,posible_contraclave=cifradoVigenere('', posible_clave, abc)
print posible_contraclave

UJSVM


In [181]:
solucion,c=cifradoVigenere(encrVigenere, posible_contraclave, abc)
print solucion

RINCONETE Y CORTADILLO.


En la venta del Molinillo, que esta puesta en los fines de los famosos
campos de Alcudia, como vamos de Castilla a la Andalucia, un dia de los
calorosos del verano se hallaron en ella acaso dos muchachos de hasta
edad de catorce a quince años el uno, y el otro no pasaba de diez y
siete: ambos de buena gracia, pero muy descosidos, rotos y maltratados:
capa no la tenian, los calzones eran de lienzo, y las medias de carne;
bien es verdad que lo enmendaban los zapatos, porque los del uno eran
alpargates tan traidos como llevados, y los del otro picados y sin
suelas, de manera que mas le servian de cormas, que de zapatos: traia
el uno montera verde de cazador, el otro un sombrero sin toquilla, bajo
de copa y ancho de falda: a la espalda, y ceñida por los pechos, traia
uno una camisa de color de camuza, encerrada y recogida toda en una
manga: el otro venia escueto y sin alforjas, puesto que en el seno se
le parecia un gran bulto, que a lo que despues parecio, era un

### Posibles atajos
Con los datos disponibles se puede llegar al texto original, pero se permite acudir a diferentes ayudas en las celdas que siguen, con la **penalización** sobre la calificación que se indica. Cada ayuda precisa de un fichero que se encuentra en una sección de la plataforma moodle. **IMPORTANTE: El mero hecho de pinchar en la tarea con la ayuda, se considerará como ayuda utilizada, incluidas las anteriores.**

- Ayuda 1, **penalización 0,5 puntos**: se da la longitud de los bloques para aplicar RSA. Descargar el fichero ${\tt lbloques.py}$

In [ ]:
##Descomentar y ejecutar
#load('lbloques.py')
#print(mensaje)

- Ayuda 2, **penalización 0,5 puntos**: no se sabe qué hacer para deshacer RSA y recuperar el resultado de hacer bloques desde el texto encriptado. Descargar el fichero ${\tt bloques.py}$ que contiene la lista de bloques ${\tt Lbloques}$

In [ ]:
##Descomentar y ejecutar
#load('bloques.py')
#len(Lbloques)

- Ayuda 3, **penalización 1 punto**: no se sabe cómo recuperar el texto encriptado a partir de los datos que nos aporta la lista de bloques. Descargar el fichero ${\tt cifrado.py}$ que contiene la cadena ${\tt encriptado}$, resultado de hacer Vigenère sobre el texto original.

In [ ]:
##Descomentar y ejecutar
#load('cifrado.py')
#len(encriptado)

- Ayuda 4, **penalización 1 punto**: no se sabe analizar la cadena **encriptado** para averiguar la longitud de la clave que se utilizó para aplicar el método de Vigenère. Descargar el fichero ${\tt lVclave.py}$ que contiene la longitud de la clave ${\tt lc}$

In [ ]:
##Descomentar y ejecutar
#load('lVclave.py')
#print('La longitud de la clave es %d'%lc)

- Ayuda 5, **penalización 2 puntos**: se da la clave de Vigenère. Descargar el fichero ${\tt Vclave.py}$ que contiene la cadena clave ${\tt Vclave}$

In [ ]:
##Descomentar y ejecutar
#load('Vclave.py')
#print(Vclave)